In [0]:
/* Not Needed
def getReachableSquares(grid : Array[Array[Char]], 
                        characters : ListBuffer[Character], 
                        character : Character) : Set[(Int, Int)] = {
    val newGrid = grid.map(_.clone)
    characters.foreach(character => {
        newGrid(character.getY)(character.getX) = character.getCharacterType})
    var c = character.getCoords
    var seenSquares = Set[(Int, Int)](c)
    var neighbouringSquares = directionsMap.values.to[Queue].map({case (x, y) => (x+c._1, y+c._2)})
        .filter(!seenSquares.contains(_))
        .filter({case (x, y) => newGrid(y)(x) == '.'})
    while (neighbouringSquares.length != 0) {
        c = neighbouringSquares.dequeue
        seenSquares += c
        neighbouringSquares ++= directionsMap.values.to[Queue].map({case (x, y) => (x+c._1, y+c._2)})
            .filter(!seenSquares.contains(_))
            .filter({case (x, y) => newGrid(y)(x) == '.'})
    }
    seenSquares -= character.getCoords
}
*/

In [0]:
/* Old method
val charTypes = Array('G', 'E')
def getShortestPathDistances(coords : (Int, Int)) : Map[(Int, Int) = {
    val newGrid = drawCharsOnGrid

    var distanceMap = Map[(Int, Int), Int](coords->0)
    var seenSquares = Set[(Int, Int)](coords)
    var neighbouringSquares = directionsMap.values.to[Queue].map({case (x, y) => (x+coords._1, y+coords._2)})
        .filter(!seenSquares.contains(_))
        .filter({case (x, y) => newGrid(y)(x) == '.'})
        .map(x => (x, 1))
    println("Initial neighbouringSquares: ", neighbouringSquares)
    while (neighbouringSquares.length != 0) {
        val (c, d) = neighbouringSquares.dequeue
        println("Next square: ", c)
        println("Distance: ", d)
        if (!seenSquares.contains(c)) {
            seenSquares += c 
            distanceMap += (c->d)
            neighbouringSquares ++= directionsMap.values.to[Queue].map({case (x, y) => (x+c._1, y+c._2)})
                .filter(!seenSquares.contains(_))
                .filter({case (x, y) => newGrid(y)(x) == '.'})
                .map(x => (x, d+1))
            println("New neighbouringSquares: ", neighbouringSquares)
        }
    }
    distanceMap
}
*/

In [0]:
/* Not any better
def recurseShortestPath(grid : Array[Array[Char]], 
                 seenSquares : Set[(Int, Int)], 
                 coords : (Int, Int),
                 distance : Int) : Map[(Int, Int), Int] = {
    val thisSquareMap = Map(coords->distance)
    val neighbouringSquares = directionsMap.values.to[Set].map({case (x, y) => (x+coords._1, y+coords._2)})
        .filter(!seenSquares.contains(_))
        .filter({case (x, y) => grid(y)(x) == '.'})
    if (neighbouringSquares.size == 0 || distance == 30) {
        return thisSquareMap
    }
    else {
        return thisSquareMap ++ neighbouringSquares.par.flatMap(x => 
                    recurseShortestPath(grid, seenSquares ++ neighbouringSquares, x, distance + 1)).toMap
    }
}
*/

In [0]:
/* Old
val character = characters(0)
val targetCharacters = characters.filter(c => c.getCharacterType != character.getCharacterType)
val neighbouringTargets = targetCharacters
      .filter(c => distance(character.getCoords, c.getCoords) == 1)
      .sorted
// Nearest character
targetCharacters.groupBy(c => distance(character.getCoords, c.getCoords)).toSeq.sortBy(_._1).head._2.sorted.head
val reachableSquares = getShortestPathDistances(grid, characters, character.getCoords)
val reachableTargetNeighbours = reachableSquares.keySet.intersect(
    targetCharacters.flatMap(c => directionsMap.values.to[Set].map({case (x, y) => (x+c.getX, y+c.getY)})).toSet)
val chosenSquare = reachableSquares.filter(x => reachableTargetNeighbours.contains(x._1)).toSeq.sortBy({case ((x,y),d) => (d, y, x)}).head._1
val neighbours = reachableSquares.filter(x => x._2 == 1)
val squareToMoveTo = getShortestPathDistances(grid, characters, chosenSquare).filter(x => neighbours.contains(x._1)).toSeq.sortBy({case ((x,y),d) => (d, y, x)}).head._1
*/

In [1]:
import scala.collection.mutable.ListBuffer
import scala.collection.mutable.Queue
import scala.collection.mutable.Set
val charTypes = Array('G', 'E')
val directions = Set((-1,0), (0,1), (1,0), (0,-1))

import scala.collection.mutable.ListBuffer
import scala.collection.mutable.Queue
import scala.collection.mutable.Set
charTypes: Array[Char] = Array('G', 'E')
directions: collection.mutable.Set[(Int, Int)] = Set(
  (1, 0),
  (0, 1),
  (0, -1),
  (-1, 0)
)

In [2]:
// Create a class for the characters
class Character extends Ordered[Character] {
    // These are backwards in tuples because of 2D Array indexing/reading
    private var posX = 0
    private var posY = 0
    private var hitPoints = 200
    private var attackPower = 3
    private var characterType = ' '
    private var alive = true
    
    def this(posX: Int, posY: Int, characterType: Char) = {
        this();
        this.posX = posX;
        this.posY = posY;
        this.characterType = characterType;
    }
    def this(posX: Int, posY: Int, characterType: Char, attackPower: Int) = {
        this();
        this.posX = posX;
        this.posY = posY;
        this.characterType = characterType;
        this.attackPower = attackPower;
    }
    
    // Update the current position
    def move(coord : (Int, Int)) : Unit = {
        posX = coord._1
        posY = coord._2
    }
    
    // Reduce the hitpoints
    def takeDamage(attackPower: Int) : Unit = {
        hitPoints -= attackPower
        if (hitPoints <= 0) {
            alive = false
        }
    }
    
    override def toString(): String = {
        s"$characterType - Remaining health: $hitPoints . Currently at ($posX, $posY)"
    }
    
    // Comparison for ordering the characters
    override def compare(that: Character): Int = {
        if (posY == that.getY) 
             posX - that.getX else
             posY - that.getY
    }
    
    def getX() : Int = {posX}
    def getY() : Int = {posY}
    def getCoords() : (Int, Int) = {(posX, posY)}
    def getAttackPower() : Int = {attackPower}
    def incrementAttackPower() : Unit = {attackPower += 1}
    def getHitPoints() : Int = {hitPoints}
    def getCharacterType() : Char = {characterType}
    def isAlive() : Boolean = {alive}
    
    def copy() : Character = {new Character(posX, posY, characterType, attackPower)}
}

defined class Character

In [3]:
import scala.math.abs
def distance(c1: (Int, Int), c2: (Int, Int)) : Int = {
    abs(c1._1 - c2._1) + abs(c1._2 - c2._2)
}

import scala.math.abs
defined function distance

In [9]:
/* Setup */
// Read in the grid
import scala.io.Source

val filename = "input"
val grid = Source.fromFile(filename).getLines.toArray.map(_ toCharArray)
var rounds = 0

// Store the characters in a structure and initialise their positions/HP
// Remap the grid to remove the characters and replace them with spaces
val inputCharacters = ListBuffer[Character]()
for {
    y <- 0 until grid.length
    x <- 0 until grid(0).length } {
    val gridChar = grid(y)(x)
    if (charTypes contains gridChar) {
        inputCharacters += new Character(x, y, gridChar)
        grid(y)(x) = '.'
    }
}
var characters = inputCharacters.map(_.copy)

// Returns a grid with the characters' positions filled in
def drawCharsOnGrid() : Array[Array[Char]] = {
    val newGrid = grid.map(_.clone)
    characters.foreach(character => {
        newGrid(character.getY)(character.getX) = character.getCharacterType})
    newGrid
}

def printGrid() : Unit = {
    val newGrid = drawCharsOnGrid
    for (row <- newGrid) {
        for (rowChar <- row) {
            print(rowChar)
        }
        println()
    }
}

// Returns a set of the neighbouring squares that are not occupied/walls
def neighbours(coords : (Int, Int)) : Set[(Int, Int)] = {
    val newGrid = drawCharsOnGrid
    directions.map({case (x, y) => (x+coords._1, y+coords._2)})
        .filter({case (x, y) => newGrid(y)(x) == '.'})
}

import scala.io.Source
filename: String = "input"
grid: Array[Array[Char]] = Array(
  Array(
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
    '#',
...
rounds: Int = 0
inputCharacters: ListBuffer[Character] = ListBuffer(
  G - Remaining health: 200 . Currently at (10, 2),
  G - Remaining health: 200 . Currently at (14, 3),
  G - Remaining health: 200 . Currently at (15, 6),
  G - Remaining health: 200 . Currently at (17, 8),
  G - Remaining health: 200 . Currently at (17, 9),
  G - Remaining health: 200 . Currently at (4, 10),
  G - Remaining health: 200 . Currently at (5, 10),
  G - Remaining health: 200 . Currently at (2, 11),
  G - Remaining health: 200 . Currently at (4, 12),
  G - Remaining health: 200 . Currently at (6, 12),
  G - Remaining health: 200 . Currently at (12, 12),
  E - Remaining health: 200 . Currently at (22, 12),
  G - Remaining health: 200 . Currently at (26, 12),
  G - Remaining health: 200 . Currentl

In [10]:
def moveTowardsNearestTarget(character : Character,
                 targetCoords : ListBuffer[(Int, Int)]) : Unit = {
    val newGrid = drawCharsOnGrid
    
    var targetDistance = -1
    val targetNeighbours = targetCoords.flatMap(neighbours)
    val potentialChosenSquares = ListBuffer[(Int, Int)]()
    var currentCoord = character.getCoords
    var currentDistance = 1
    var distanceMap = Map[(Int, Int), Int](currentCoord->0)
    var coordNeighbours = neighbours(currentCoord) 
    var squaresToCheck = coordNeighbours.to[Queue]
    distanceMap ++= coordNeighbours.map(neighbour => (neighbour, currentDistance))
    
    // BFS - Move out to each of the neighbours.
    // Iteratively do this until on of the target squares is hit.
    // At this point, keep going until the distance becomes the target distance +1
    while (squaresToCheck.length != 0 && currentDistance != targetDistance + 1) {
        currentCoord = squaresToCheck.dequeue
        currentDistance = distanceMap(currentCoord)
        if (currentDistance != targetDistance + 1) {
            if (targetNeighbours.contains(currentCoord)) {
                targetDistance = currentDistance
                potentialChosenSquares += currentCoord
            }
            else {
                coordNeighbours = neighbours(currentCoord)
                    .filter(neighbour => !distanceMap.keySet.contains(neighbour))
                squaresToCheck ++= coordNeighbours
                distanceMap ++= coordNeighbours.map(neighbour => (neighbour, currentDistance+1))
            }
        }
    }
    
    // Choose a square based on reading order (if one was found).
    if (targetDistance != -1) {
        val chosenSquare = potentialChosenSquares.sortBy({case (x,y) => (y, x)}).head
        currentDistance = targetDistance
        var nextSteps = Set[(Int, Int)](chosenSquare)
        // Propagate back to the original square following the neighbours in the map that have d-1 iteratively.
        while(currentDistance != 1) {
            nextSteps = nextSteps.flatMap(neighbours _)
                .filter(neighbour => distanceMap.contains(neighbour) && distanceMap(neighbour) == currentDistance-1)
            currentDistance -= 1
        }
        character.move(nextSteps.toSeq.sortBy({case (x,y) => (y, x)}).head)
    }
}

defined function moveTowardsNearestTarget

In [11]:
//println("Rounds: ", rounds)
//characters.foreach(println)
//printGrid
while (characters.filter(c => c.getCharacterType == 'G').length > 0 &&
       characters.filter(c => c.getCharacterType == 'E').length > 0) {
    var fullRoundCompleted = true
    val charactersToMove = characters.sorted
    charactersToMove.foreach(character => {
        if (character.isAlive) {
            val targetCharacters = characters.filter(c => c.getCharacterType != character.getCharacterType)
            if (targetCharacters.length == 0) {
                fullRoundCompleted = false
            }
            // Check if there are any neighbouring targets
            var neighbouringTargets = targetCharacters
              .filter(c => distance(character.getCoords, c.getCoords) == 1)
              .sortBy(c => (c.getHitPoints, c.getY, c.getX))
            if (neighbouringTargets.length == 0) {
                // Move towards the nearest reachable square
                moveTowardsNearestTarget(character, targetCharacters.map(_ getCoords))
                neighbouringTargets = targetCharacters
                  .filter(c => distance(character.getCoords, c.getCoords) == 1)
                  .sortBy(c => (c.getHitPoints, c.getY, c.getX))
            }
            // Redetermine whether we should attack
            if (neighbouringTargets.length >= 1) {
                // Attack the first target in neighbouring targets
                val target = neighbouringTargets.head
                target.takeDamage(character.getAttackPower)
                if (!target.isAlive) {  
                    characters -= target
                }
            }
        }
    })
    if (fullRoundCompleted) {
      rounds += 1
    }
    println("\rRounds: " + rounds + ", numChars: " + characters.length)
    //printGrid
    //characters.foreach(println)
}
//rounds * characters.map(_ getHitPoints).sum

Rounds: 1, numChars: 30
Rounds: 2, numChars: 30
Rounds: 3, numChars: 30
Rounds: 4, numChars: 30
Rounds: 5, numChars: 30
Rounds: 6, numChars: 30
Rounds: 7, numChars: 30
Rounds: 8, numChars: 30
Rounds: 9, numChars: 30
Rounds: 10, numChars: 30
Rounds: 11, numChars: 30
Rounds: 12, numChars: 30
Rounds: 13, numChars: 30
Rounds: 14, numChars: 30
Rounds: 15, numChars: 30
Rounds: 16, numChars: 30
Rounds: 17, numChars: 30
Rounds: 18, numChars: 30
Rounds: 19, numChars: 30
Rounds: 20, numChars: 30
Rounds: 21, numChars: 30
Rounds: 22, numChars: 30
Rounds: 23, numChars: 30
Rounds: 24, numChars: 30
Rounds: 25, numChars: 30
Rounds: 26, numChars: 30
Rounds: 27, numChars: 29
Rounds: 28, numChars: 29
Rounds: 29, numChars: 28
Rounds: 30, numChars: 28
Rounds: 31, numChars: 28
Rounds: 32, numChars: 28
Rounds: 33, numChars: 28
Rounds: 34, numChars: 28
Rounds: 35, numChars: 28
Rounds: 36, numChars: 28
Rounds: 37, numChars: 28
Rounds: 38, numChars: 28
Rounds: 39, numChars: 27
Rounds: 40, numChars: 27
Rounds: 4

In [12]:
println("Rounds: ", rounds)
characters.foreach(println)
printGrid
characters.map(_ getHitPoints).sum
rounds * characters.map(_ getHitPoints).sum


(Rounds: ,107)
G - Remaining health: 191 . Currently at (18, 22)
G - Remaining health: 200 . Currently at (23, 14)
G - Remaining health: 143 . Currently at (21, 15)
G - Remaining health: 194 . Currently at (22, 13)
G - Remaining health: 200 . Currently at (19, 23)
G - Remaining health: 200 . Currently at (20, 11)
G - Remaining health: 200 . Currently at (17, 23)
G - Remaining health: 200 . Currently at (18, 21)
G - Remaining health: 200 . Currently at (22, 12)
G - Remaining health: 200 . Currently at (11, 16)
G - Remaining health: 77 . Currently at (12, 11)
G - Remaining health: 200 . Currently at (14, 10)
G - Remaining health: 137 . Currently at (10, 13)
################################
##############.#################
##########.##....###############
#########.......################
#########...........############
#########...........############
##########.........#############
###########.........############
########.#.#..#.......##########
#######.................########
##....

res11_3: Int = 2342
res11_4: Int = 250594

## Part 2

In [13]:
//println("Rounds: ", rounds)
//characters.foreach(println)
//printGrid
var elvesDied = true
var attackPower = 3
while (elvesDied) {
    inputCharacters.filter(c => c.getCharacterType == 'E').foreach(_ incrementAttackPower)
    characters = inputCharacters.map(_.copy)
    rounds = 0
    attackPower += 1
    while (characters.filter(c => c.getCharacterType == 'G').length > 0 &&
           characters.filter(c => c.getCharacterType == 'E').length > 0) {
        var fullRoundCompleted = true
        val charactersToMove = characters.sorted
        charactersToMove.foreach(character => {
            if (character.isAlive) {
                val targetCharacters = characters.filter(c => c.getCharacterType != character.getCharacterType)
                if (targetCharacters.length == 0) {
                    fullRoundCompleted = false
                }
                // Check if there are any neighbouring targets
                var neighbouringTargets = targetCharacters
                  .filter(c => distance(character.getCoords, c.getCoords) == 1)
                  .sortBy(c => (c.getHitPoints, c.getY, c.getX))
                if (neighbouringTargets.length == 0) {
                    // Move towards the nearest reachable square
                    moveTowardsNearestTarget(character, targetCharacters.map(_ getCoords))
                    neighbouringTargets = targetCharacters
                      .filter(c => distance(character.getCoords, c.getCoords) == 1)
                      .sortBy(c => (c.getHitPoints, c.getY, c.getX))
                }
                // Redetermine whether we should attack
                if (neighbouringTargets.length >= 1) {
                    // Attack the first target in neighbouring targets
                    val target = neighbouringTargets.head
                    target.takeDamage(character.getAttackPower)
                    if (!target.isAlive) {  
                        characters -= target
                    }
                }
            }
        })
        if (fullRoundCompleted) {
          rounds += 1
        }
        println("\rRounds: " + rounds + ", numChars: " + characters.length)
        //printGrid
        //characters.foreach(println)
    }
    elvesDied = characters.filter(c => c.getCharacterType == 'E').length < 
                inputCharacters.filter(c => c.getCharacterType == 'E').length
    println("\rAttackPower: " + attackPower)
}
rounds * characters.map(_ getHitPoints).sum

Rounds: 1, numChars: 30
Rounds: 2, numChars: 30
Rounds: 3, numChars: 30
Rounds: 4, numChars: 30
Rounds: 5, numChars: 30
Rounds: 6, numChars: 30
Rounds: 7, numChars: 30
Rounds: 8, numChars: 30
Rounds: 9, numChars: 30
Rounds: 10, numChars: 30
Rounds: 11, numChars: 30
Rounds: 12, numChars: 30
Rounds: 13, numChars: 30
Rounds: 14, numChars: 30
Rounds: 15, numChars: 30
Rounds: 16, numChars: 30
Rounds: 17, numChars: 30
Rounds: 18, numChars: 30
Rounds: 19, numChars: 30
Rounds: 20, numChars: 30
Rounds: 21, numChars: 29
Rounds: 22, numChars: 29
Rounds: 23, numChars: 29
Rounds: 24, numChars: 29
Rounds: 25, numChars: 29
Rounds: 26, numChars: 29
Rounds: 27, numChars: 29
Rounds: 28, numChars: 29
Rounds: 29, numChars: 28
Rounds: 30, numChars: 27
Rounds: 31, numChars: 27
Rounds: 32, numChars: 27
Rounds: 33, numChars: 27
Rounds: 34, numChars: 27
Rounds: 35, numChars: 27
Rounds: 36, numChars: 27
Rounds: 37, numChars: 27
Rounds: 38, numChars: 27
Rounds: 39, numChars: 27
Rounds: 40, numChars: 26
Rounds: 4

Rounds: 3, numChars: 30
Rounds: 4, numChars: 30
Rounds: 5, numChars: 30
Rounds: 6, numChars: 30
Rounds: 7, numChars: 30
Rounds: 8, numChars: 30
Rounds: 9, numChars: 30
Rounds: 10, numChars: 30
Rounds: 11, numChars: 30
Rounds: 12, numChars: 30
Rounds: 13, numChars: 30
Rounds: 14, numChars: 29
Rounds: 15, numChars: 29
Rounds: 16, numChars: 29
Rounds: 17, numChars: 29
Rounds: 18, numChars: 29
Rounds: 19, numChars: 29
Rounds: 20, numChars: 28
Rounds: 21, numChars: 28
Rounds: 22, numChars: 28
Rounds: 23, numChars: 28
Rounds: 24, numChars: 28
Rounds: 25, numChars: 28
Rounds: 26, numChars: 27
Rounds: 27, numChars: 27
Rounds: 28, numChars: 27
Rounds: 29, numChars: 26
Rounds: 30, numChars: 26
Rounds: 31, numChars: 25
Rounds: 32, numChars: 25
Rounds: 33, numChars: 25
Rounds: 34, numChars: 24
Rounds: 35, numChars: 23
Rounds: 36, numChars: 23
Rounds: 37, numChars: 23
Rounds: 38, numChars: 23
Rounds: 39, numChars: 23
Rounds: 40, numChars: 23
Rounds: 41, numChars: 23
Rounds: 42, numChars: 22
Rounds:

Rounds: 73, numChars: 8
Rounds: 74, numChars: 8
Rounds: 75, numChars: 8
Rounds: 76, numChars: 8
Rounds: 77, numChars: 8
Rounds: 78, numChars: 8
Rounds: 79, numChars: 8
Rounds: 80, numChars: 8
Rounds: 80, numChars: 7
AttackPower: 10
Rounds: 1, numChars: 30
Rounds: 2, numChars: 30
Rounds: 3, numChars: 30
Rounds: 4, numChars: 30
Rounds: 5, numChars: 30
Rounds: 6, numChars: 30
Rounds: 7, numChars: 30
Rounds: 8, numChars: 30
Rounds: 9, numChars: 30
Rounds: 10, numChars: 30
Rounds: 11, numChars: 29
Rounds: 12, numChars: 29
Rounds: 13, numChars: 29
Rounds: 14, numChars: 29
Rounds: 15, numChars: 28
Rounds: 16, numChars: 28
Rounds: 17, numChars: 28
Rounds: 18, numChars: 28
Rounds: 19, numChars: 28
Rounds: 20, numChars: 27
Rounds: 21, numChars: 27
Rounds: 22, numChars: 27
Rounds: 23, numChars: 26
Rounds: 24, numChars: 25
Rounds: 25, numChars: 25
Rounds: 26, numChars: 22
Rounds: 27, numChars: 22
Rounds: 28, numChars: 22
Rounds: 29, numChars: 22
Rounds: 30, numChars: 22
Rounds: 31, numChars: 21
Ro

Rounds: 3, numChars: 30
Rounds: 4, numChars: 30
Rounds: 5, numChars: 30
Rounds: 6, numChars: 30
Rounds: 7, numChars: 30
Rounds: 8, numChars: 30
Rounds: 9, numChars: 29
Rounds: 10, numChars: 29
Rounds: 11, numChars: 29
Rounds: 12, numChars: 28
Rounds: 13, numChars: 28
Rounds: 14, numChars: 28
Rounds: 15, numChars: 28
Rounds: 16, numChars: 27
Rounds: 17, numChars: 26
Rounds: 18, numChars: 26
Rounds: 19, numChars: 25
Rounds: 20, numChars: 24
Rounds: 21, numChars: 22
Rounds: 22, numChars: 22
Rounds: 23, numChars: 22
Rounds: 24, numChars: 22
Rounds: 25, numChars: 22
Rounds: 26, numChars: 22
Rounds: 27, numChars: 21
Rounds: 28, numChars: 20
Rounds: 29, numChars: 20
Rounds: 30, numChars: 19
Rounds: 31, numChars: 19
Rounds: 32, numChars: 16
Rounds: 33, numChars: 16
Rounds: 34, numChars: 16
Rounds: 35, numChars: 15
Rounds: 36, numChars: 15
Rounds: 37, numChars: 15
Rounds: 38, numChars: 15
Rounds: 39, numChars: 14
Rounds: 40, numChars: 13
Rounds: 41, numChars: 12
Rounds: 42, numChars: 12
Rounds:

elvesDied: Boolean = false
attackPower: Int = 19
res12_3: Int = 52133

In [14]:
characters

res13: ListBuffer[Character] = ListBuffer(
  E - Remaining health: 86 . Currently at (18, 11),
  E - Remaining health: 125 . Currently at (20, 11),
  E - Remaining health: 167 . Currently at (19, 9),
  E - Remaining health: 158 . Currently at (13, 21),
  E - Remaining health: 176 . Currently at (19, 10),
  E - Remaining health: 137 . Currently at (12, 22),
  E - Remaining health: 200 . Currently at (21, 14),
  E - Remaining health: 200 . Currently at (13, 22),
  E - Remaining health: 5 . Currently at (10, 20),
  E - Remaining health: 155 . Currently at (11, 21)
)